<a href="https://colab.research.google.com/github/JKSmirny/hw_22.big-data-challenge.amazon_reviews.amazone.vine/blob/main/JKS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (86.9 kB/s)
Reading package li

# New Section

In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-22 20:00:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-05-22 20:00:40 (5.93 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReviewsETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_multilingual_FR_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         FR|      14952|R32VYUWDIB5LKE|0552774294|     362925721|    The God Delusion|               Books|          5|            0|          0|   N|                Y|a propos de ce livre|je conseille fort...| 2013-02-13|
|         FR|      14952|R3CCMP4EV6HAVL|B004GJXQ20|     268067011|A Game of Thrones...|Digital_Ebook_Pur

In [8]:
print(df.count())

254080


In [9]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
products = df.select(["product_id", "product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0552774294|    The God Delusion|
|B004GJXQ20|A Game of Thrones...|
|B00GIGGS6A|      Huion H610 PRO|
|B00CW7KK9K|Withings Pulse - ...|
|B002L6SKIK|          Prometheus|
+----------+--------------------+
only showing top 5 rows



In [11]:
print(products.count())

254080


In [12]:
products = products.dropDuplicates()
print(products.count())

35451


In [13]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
#df_custdata = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = df.groupby("customer_id").count()
customers = customers_df.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      26425|             1|
|      29814|             1|
|      78440|             2|
|      90550|             1|
|      99817|             1|
|    2846240|             1|
|    2966808|             1|
|    3323007|             1|
|    3458442|             1|
|    3497479|             1|
|    3498767|             1|
|    3511947|             1|
|    3945593|             1|
|    4049807|             1|
|    4214333|             1|
|    4262489|             2|
|    4423902|             1|
|    4660180|             2|
|    4667529|             3|
|    4838481|             2|
+-----------+--------------+
only showing top 20 rows



In [15]:
print(customers.count())

131386


In [16]:
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [17]:
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R32VYUWDIB5LKE|      14952|0552774294|     362925721| 2013-02-13|
|R3CCMP4EV6HAVL|      14952|B004GJXQ20|     268067011| 2014-08-03|
|R14NAE6UGTVTA2|      17564|B00GIGGS6A|     256731097| 2015-07-07|
|R2E7QEWSC6EWFA|      18940|B00CW7KK9K|     977480037| 2014-06-16|
|R26E6I47GQRYKR|      20315|B002L6SKIK|     827187473| 2013-06-10|
|R1RJMTSNCKB9LP|      20842|B00004YLIU|     678427290| 2013-04-15|
|R2P2XF84YELQBZ|      20913|B00AYHK7RU|     108403123| 2013-06-07|
| RJKULSX2Y5R07|      21490|B00CJ3V5UK|     252503117| 2014-11-07|
|R3UYE0U7SQCI8Q|      24196|B000FUM0TE|      48021829| 2012-11-16|
|R1TKJ7XFS3RDEB|      24196|B000UMXCTY|     181554537| 2013-04-23|
|R3S9JNS21QDBXP|      24196|B0044JV1K6|     386772628| 2013-08-02|
|R2Y4O06QMOGHD0|      24196|2226249672|     753674225| 2013-10

In [18]:
print(review_id_table.count())

254080


In [19]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [37]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType,BooleanType,DateType
df2 = df.withColumn("review_id",col("review_id").cast(StringType())) \
    .withColumn("customer_id",col("customer_id").cast(IntegerType())) \
    .withColumn("product_id",col("product_id").cast(StringType())) \
    .withColumn("product_parent",col("product_parent").cast(IntegerType())) \
    .withColumn("review_date",col("review_date").cast(DateType()))
df2.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [38]:
review_id_table = df2.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R32VYUWDIB5LKE|      14952|0552774294|     362925721| 2013-02-13|
|R3CCMP4EV6HAVL|      14952|B004GJXQ20|     268067011| 2014-08-03|
|R14NAE6UGTVTA2|      17564|B00GIGGS6A|     256731097| 2015-07-07|
|R2E7QEWSC6EWFA|      18940|B00CW7KK9K|     977480037| 2014-06-16|
|R26E6I47GQRYKR|      20315|B002L6SKIK|     827187473| 2013-06-10|
|R1RJMTSNCKB9LP|      20842|B00004YLIU|     678427290| 2013-04-15|
|R2P2XF84YELQBZ|      20913|B00AYHK7RU|     108403123| 2013-06-07|
| RJKULSX2Y5R07|      21490|B00CJ3V5UK|     252503117| 2014-11-07|
|R3UYE0U7SQCI8Q|      24196|B000FUM0TE|      48021829| 2012-11-16|
|R1TKJ7XFS3RDEB|      24196|B000UMXCTY|     181554537| 2013-04-23|
|R3S9JNS21QDBXP|      24196|B0044JV1K6|     386772628| 2013-08-02|
|R2Y4O06QMOGHD0|      24196|2226249672|     753674225| 2013-10

In [40]:
print(review_id_table.count())

254080


In [39]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [21]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R32VYUWDIB5LKE|          5|            0|          0|   N|
|R3CCMP4EV6HAVL|          5|            0|          0|   N|
|R14NAE6UGTVTA2|          3|            1|          3|   N|
|R2E7QEWSC6EWFA|          4|            0|          1|   N|
|R26E6I47GQRYKR|          2|            3|          5|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [22]:
print(vine_table.count())

254080


In [23]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [24]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://reviews-frtv.crql9pfluf9x.us-east-1.rds.amazonaws.com:5432/reviews-frtv"
config = {"user":"root",
          "password": "Ben&LizzyA22",
          "driver":"org.postgresql.Driver"}

In [25]:
# Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
config = {"user":"root",
          "password": "Ben&LizzyA22",
          "driver":"org.postgresql.Driver"}

In [26]:
# Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
config = {"user":"root",
          "password": "Ben&LizzyA22",
          "driver":"org.postgresql.Driver"}

In [27]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
config = {"user":"root",
          "password": "Ben&LizzyA22",
          "driver":"org.postgresql.Driver"}

In [41]:
# Write DataFrame to review_id_table table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
config = {"user":"root",
          "password": "Ben&LizzyA22",
          "driver":"org.postgresql.Driver"}